# Project Report: Income Prediction and Sentiment Analysis

## Data Preparation and Preprocessing

Pandas was used to import bank customer data into Jupyter notebook. It was done before the study. A first inspection of the data showed a column called "ID," which was unhelpful and was immediately eliminated to eliminate unnecessary information. One-hot encoding was also used to represent categorical attributes like education and account ownership. This improvement allowed regression and neural network algorithms to understand non-numerical data.

After that, the Interquartile Range (IQR) method was used to remove Mortgage(Thousands) column outliers. This step reduced noise that could disrupt model learning to ensure a cleaner and more robust training dataset. The feature distributions were normalized by scaling the data with StandardScaler. This is crucial to enhancing neural network performance, especially when features vary greatly.

The dataset was cleaned and updated before being split into training and testing sets. This allowed proper model evaluation on unreleased data.

## Regression and Neural Network Modeling

Two models were utilized in order to make a prediction regarding the Income(Thousands) variable. These models included a feedforward Neural Network and a conventional Linear Regression model. The linear model was used as a baseline, and it achieved a Mean Squared Error (MSE) value of 874.21. However, despite the fact that this performance was satisfactory, it brought to light shortcomings in the ability to capture complicated relationships within the data.

After that, TensorFlow and Keras were utilized in order to build a Neural Network that was advanced in its capabilities. The architecture was trained over a period of ten epochs and consisted of many dense layers that were activated by ReLU. This initial model demonstrated the value of non-linear learning capabilities by displaying a lower MSE of 785.77 when compared to the original model.

An additional phase of hyperparameter tweaking was performed with the help of a bespoke wrapper that was compatible with GridSearchCV in order to further increase performance. For this reason, it was possible to systematically optimize both the batch size and the optimizer type. The model that performed the best utilized the Adam optimizer with a batch size of 128, which resulted in a marginally improved MSE of 780.17 after the optimization process. In spite of the fact that deep learning gives advantages, the dataset may be either simple or too large to fully utilize the capabilities of neural networks, as indicated by the marginal gains.

Following careful consideration, the neural network model was selected as the ultimate predictive algorithm due to the fact that it possesses higher accuracy and versatility. Because it was able to simulate non-linear interactions, it was better equipped to capture the underlying patterns that were affecting the income of customers.

## Income Prediction for a New Customer

To demonstrate the predictive capability of the finalized neural network model, a hypothetical customer profile was constructed. The profile was based on a 45-year-old individual with 20 years of experience, a mortgage of $150,000, a credit score of 700, and other personal and banking features set appropriately.

This new data point was processed using the same one-hot encoding structure and standardization scaler applied to the training data. When passed through the optimized neural network model, the predicted income for the customer was 13,029.01 (in thousands), or approximately $13 million, which seems extraordinarily high. This suggests potential issues with the scale or structure of the original target variable and may warrant further data validation or log transformation for better interpretability.

## Sentiment and Semantic Analysis of YouTube Comments

The project also included a semantic analysis component to examine viewer sentiment from a YouTube video about India-Pakistan conflict. A total of 100 top comments were extracted using the YoutubeCommentDownloader library. The text data underwent thorough cleaning, including removing URLs, punctuation, digits, and converting text to lowercase for uniformity.

Using the TextBlob library, the comments were classified into three sentiment categories: positive, neutral, and negative. Based on the analysis, it was determined that the bulk of the comments were neither positive nor negative, followed by neutral comments. This shows that the viewers were primarily passive or informational in tone, rather than emotionally charged, during the presentation, despite the fact that the subject matter was sensitive.

In addition to this, an emotion classification was carried out with the help of a HuggingFace transformer model that had been pre-trained (j-hartmann/emotion-english-distilroberta-base). Despite the fact that certain comments were returned with a 'unknown' classification due to their brevity or the absence of emotional markers, this model nevertheless supplied a rich emotional context that went beyond basic sentiment.

A bar chart was used to represent the distribution of sentiment, and it was able to clearly demonstrate that neutral sentiment was the predominant sentiment. A word cloud was also created in order to illustrate the words that were used the most frequently throughout all of the comments. The political and conflict-related aspect of the video content was reflected in the use of notable terms such as "war," "india," "pakistan," "modi," and "attack." These particular words contribute to the reinforcement of the context of the conversation as well as the types of feelings that are likely to be expressed.



## Conclusion and Final Thoughts

This project combined structured income prediction and unstructured sentiment analysis to explore both numerical and textual data. In the regression task, while linear regression provided a useful baseline, the neural network outperformed it in terms of MSE and adaptability. The income prediction for a new customer further validated the model's applicability in a real-world scenario, although scale issues may need further investigation.

On the semantic analysis front, the integration of classical sentiment scoring and transformer-based emotion recognition allowed for a rich understanding of viewer opinions and emotional tone. Visualization techniques such as bar charts and word clouds helped to summarize the findings effectively.

# PROJECT CODE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
data = pd.read_csv("/content/BankRecords.csv")
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data = data.drop(['ID'], axis = 1)

In [ ]:
# Identify categorical columns
categorical_cols = data.select_dtypes(include=['object', 'category']).columns

# Print unique values in each categorical column
for col in categorical_cols:
    print(f"Unique values in '{col}':")
    print(data[col].unique())
    print()

In [ ]:
# One-hot encode categorical columns
df_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=False)

# Preview the resulting dataframe
df_encoded.head()

In [ ]:
df_encoded.describe()

In [ ]:
# Rename columns
df_encoded = df_encoded.rename(columns={
    'Experience(Years)': 'Experience_Years',
    'Income(Thousands\'s)': 'Income_Thousands',
    'Mortgage(Thousands\'s)': 'Mortgage_Thousands'
})


In [ ]:
# Calculate Q1 and Q3 for the 'mortgage' column
Q1 = df_encoded['Mortgage_Thousands'].quantile(0.25)
Q3 = df_encoded['Mortgage_Thousands'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the dataframe to remove outliers
df_no_outliers = df_encoded[(df_encoded['Mortgage_Thousands'] >= lower_bound) & (df_encoded['Mortgage_Thousands'] <= upper_bound)]

# reset index
df_no_outliers.reset_index(drop=True, inplace=True)

df_no_outliers.head()

In [ ]:
# Features and target
X = df_no_outliers.drop(['Income_Thousands'], axis = 1)  # Use features to predict income
y = df_no_outliers['Income_Thousands']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for Neural Network)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Initialize and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluate the model
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Linear Regression Mean Squared Error: {mse_lr}")


In [ ]:
# Initialize the Neural Network model
nn_model = Sequential()
nn_model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(1))  # Single output for regression

# Compile the model
nn_model.compile(optimizer='rmsprop', loss='mean_squared_error')

# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=10, batch_size=16, validation_data=(X_test_scaled, y_test), verbose=1)

# Predict on the test set
y_pred_nn = nn_model.predict(X_test_scaled)

# Evaluate the model
mse_nn = mean_squared_error(y_test, y_pred_nn)
print(f"Neural Network Mean Squared Error: {mse_nn}")


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error

# Define the function to create the neural network model
def create_nn_model(optimizer='adam', neurons=64):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Create a custom scikit-learn wrapper class for Keras
class KerasRegressorCustom(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn=None, optimizer='adam', neurons=64, epochs=100, batch_size=32, verbose=0):
        self.build_fn = build_fn
        self.optimizer = optimizer
        self.neurons = neurons
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose

    def fit(self, X, y):
        self.model = self.build_fn(optimizer=self.optimizer, neurons=self.neurons)
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Wrap the model with KerasRegressorCustom for use in GridSearchCV
nn_model = KerasRegressorCustom(build_fn=create_nn_model, epochs=100, batch_size=32, verbose=0)

# Define the parameter grid for GridSearchCV
param_grid_nn = {
    'optimizer': ['adam', 'rmsprop'],  # Optimizer types
    'batch_size': [64, 128],  # Batch size
}

# Perform GridSearchCV with cross-validation
grid_search_nn = GridSearchCV(estimator=nn_model, param_grid=param_grid_nn,
                              cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search_nn.fit(X_train_scaled, y_train)

# Best parameters and model performance
print(f"Best hyperparameters for Neural Network: {grid_search_nn.best_params_}")
best_nn_model = grid_search_nn.best_estimator_

# Predict on the test set
y_pred_nn = best_nn_model.predict(X_test_scaled)

# Evaluate the model
mse_nn = mean_squared_error(y_test, y_pred_nn)
print(f"Neural Network Mean Squared Error: {mse_nn}")


In [ ]:
new_customer = {
    'Age': 45,
    'Experience_Years': 20,
    'Sort Code': 1010,
    'Family': 3,
    'Credit Score': 700,
    'Mortgage_Thousands': 150,
    'Education_Degree': 1,
    'Education_Diploma': 0,
    'Education_Masters': 0,
    'Personal Loan_No': 0,
    'Personal Loan_Yes': 1,
    'Securities Account_No': 1,
    'Securities Account_Yes': 0,
    'CD Account_No': 1,
    'CD Account_Yes': 0,
    'Online Banking_No': 0,
    'Online Banking_Yes': 1,
    'CreditCard_No': 0,
    'CreditCard_Yes': 1
}

# Convert to DataFrame
new_customer_df = pd.DataFrame([new_customer])

# Apply the same scaler used on the training data
new_customer_scaled = scaler.transform(new_customer_df)

predicted_income = best_nn_model.predict(new_customer_scaled)
print(f"Predicted Income (in Thousands): {predicted_income[0][0]:.2f}")


## Semantic Analysis

In [ ]:
from itertools import islice
from youtube_comment_downloader import *
import pandas as pd

# Initialize the downloader
downloader = YoutubeCommentDownloader()

# Fetch top 10 popular comments
comments = downloader.get_comments_from_url(
    'https://www.youtube.com/watch?v=PuSQPHOJXNY'
)

# Convert to list of dictionaries (each comment is a dict)
comment_list = list(islice(comments, 100))

# Create DataFrame
df = pd.DataFrame(comment_list)

# Display the DataFrame
print(df.head())


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
import re
import nltk
from transformers import pipeline

nltk.download('punkt')

In [ ]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # remove URLs
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # remove punctuation & digits
    text = text.lower()  # lowercase
    return text.strip()

df['clean_text'] = df['text'].astype(str).apply(clean_text)

def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return 'Positive'
    elif polarity < -0.1:
        return 'Negative'
    else:
        return 'Neutral'

df['Sentiment'] = df['clean_text'].apply(get_sentiment)

emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)

def classify_emotion(text):
    try:
        result = emotion_model(text)[0]
        return result['label']
    except:
        return 'unknown'

# Apply only to non-empty cleaned texts
df['Emotion'] = df['clean_text'].apply(lambda x: classify_emotion(x) if len(x) > 3 else 'unknown')

plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Sentiment', palette='Set2', order=['Positive', 'Neutral', 'Negative'])
plt.title("Sentiment Distribution (3 Categories)")
plt.xlabel("Sentiment")
plt.ylabel("Number of Comments")
plt.tight_layout()
plt.show()


In [ ]:
text_all = " ".join(df['clean_text'].tolist())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_all)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud of YouTube Comments")
plt.tight_layout()
plt.show()
